

---

# ♻  **Bing Indexing for Albarak { ▶ https://hallroad.pk/ }**                 

---





In [ ]:
# @title
from google.colab import auth
import requests
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Following values your data :-
api_key = 'b4735d6028f35c3'
api_password = 'shpat_'
store_url = "https://albarak-mobile.myshopify.com"
store_name = "albarak-mobile.myshopify.com"
store_front_name = "albarakmobile.com"


def fetch_id_from_webhook():
    # Set up credentials
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    credentials = ServiceAccountCredentials.from_json_keyfile_name('Service account Key.json', scope)
    gc = gspread.authorize(credentials)

    # Open the Google Sheet by its title
    spreadsheet_title = 'DataBase_WEBHOOK'
    sh = gc.open(spreadsheet_title)

    # Select the worksheet by title
    worksheet_title = 'alburak'
    worksheet = sh.worksheet(worksheet_title)

    # Get all values from the second column
    column_values = worksheet.col_values(2)[0:]  # Assuming there is not a header in the first row

    # Remove duplicates and store in a list
    distinct_values = list(set(column_values))
   # Clear all data in the worksheet
    worksheet.clear()

    # Create a list of Cell objects
    cells = worksheet.range('B1:B{}'.format(len(distinct_values)))

    # Update each cell with the corresponding value
    for i, cell in enumerate(cells):
        cell.value = distinct_values[i]

    # Update the worksheet in a single batch
    worksheet.update_cells(cells)

    print(f"Values {distinct_values} added to the second column of the Google Sheet.")

    print("Number of items in the list:", len(distinct_values))
    if distinct_values:
        return distinct_values


def product_handle(product_id=None):
    global api_key, api_password, store_name

    try:
        # Build the URL
        url = f'https://{store_name}/admin/api/2024-01/products/{product_id}.json'

        # Set up authentication headers
        header = {
            "Content-Type": "application/json",
        }

        auth = (api_key, api_password)

        # Make the API request
        response_ = requests.get(url, headers=header, auth=auth)

        # Check if the request was successful (status code 200)
        if response_.status_code == 200:
            # Parse the JSON response
            product_data = response_.json()

            # Extract the product handle
            product_handel = product_data["product"]["handle"]
            if product_data["product"]['status'] == 'active':
                product_url = f"https://{store_front_name}/products/{product_handel}"
                print(product_url)
                return product_url
        else:
            print(f"Error: {response_.status_code} - {response_.text}")
    except requests.exceptions.RequestException as e:
        print(f"RequestException: {e}")
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")


def indexing_Urls(product_urls=None):
    if product_urls:
        global store_front_name
        headers = {
            'Content-Type': 'application/json',
        }
        data = {
            "siteUrl": f"http://www.{store_front_name}",
            "urlList": [product_urls]
        }

        response = requests.post(
            'https://ssl.bing.com/webmaster/api.svc/json/SubmitUrlbatch?apikey=66e9ecfc59024b0dfbe1fe',
            headers=headers, data=json.dumps(data))

        if response.status_code == 200:
            print('URLs submitted successfully')
            return True
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return False


def Update_database(find_id=None):
    # Replace with your Google Sheets API credentials JSON file path
    JSON_KEY_FILE = 'Service account Key.json'

    # Replace with your Google Sheets document name
    SPREADSHEET_NAME = 'DataBase_WEBHOOK'

    # Replace with your specific worksheet name
    WORKSHEET_NAME = 'alburak'

    # Authorize and authenticate with Google Sheets API
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(JSON_KEY_FILE, scope)
    gc = gspread.authorize(credentials)

    # Open the Google Sheets document by name
    spreadsheet = gc.open(SPREADSHEET_NAME)

    # Choose the specific worksheet by name
    worksheet = spreadsheet.worksheet(WORKSHEET_NAME)

    # Get all values from the second column
    column_values = worksheet.col_values(2)

    # Find the index of the value to be updated
    if find_id is not None:
        try:
            index = column_values.index(str(find_id)) + 1

            # Clear the content in cells above the found value
            worksheet.update('B1:B{}'.format(index), [[''] * 1] * (index))
            print(f"Rows above the one containing '{find_id}' in the second column have been cleared.")

            # Store all values in a list
            distinct_values = column_values[index:]

            # Clear all data in the worksheet
            worksheet.clear()
            print("Worksheet cleared.")

            # Create a list of Cell objects
            cells = worksheet.range('B1:B{}'.format(len(distinct_values)+2))

            # Update each cell with the corresponding value
            for i, cell in enumerate(cells):
                cell.value = distinct_values[i]

            # Update the worksheet in a single batch
            worksheet.update_cells(cells)
            print("Values written to column B.")

            # Save changes
            spreadsheet.save()
            print("Changes saved.")
        except ValueError:
            print(f"Special code '{find_id}' not found in the second column.")


try:

    distinct_id = fetch_id_from_webhook()

    # Print the distinct values
    # print("Distinct Ids from Webhook : ", distinct_id)
    last_id = None

    for ID in distinct_id:
        product_front_url = product_handle(ID)
        if product_front_url:
            if indexing_Urls(product_front_url):
                last_id = ID

    Update_database(last_id)

except gspread.exceptions.GSpreadException as e:
    print(f"GSpreadException: {e}")

except Exception as e:
    print(f"Unexpected error: {e}")


Values ['8885854535969', '8987904540961', '8885943238945', '8885845360929', '8939473109281', '8885830713633', '9001826222369', '8885859680545', '8885853094177', '8885860729121', '8885946220833', '8885828321569', '8885823504673', '8926616944929', '8885827076385', '8885829861665', '8885856829729', '8885889597729', '8885987770657', '8885853421857', '8885961982241', '8885852864801', '8885886779681', '8885863022881', '8885916139809', '8885866889505', '8885853716769', '8885857091873', '8885891694881', '8885849719073', '8885821341985', '8885820883233', '8994629878049', '8997492457761', '8885841297697', '8885891891489', '8885830320417', '8885863481633', '8885818523937', '8885822849313', '8885962539297', '8885856010529', '8885840281889', '8885989114145', '8885861974305', '8885849653537', '8939477467425', '8885827436833', '9005824344353', '8885880357153', '8885874196769', '9005760151841', '8885817245985', '8885818163489', '8885831532833', '8885881897249', '8885861056801', '8885852963105', '88858